### Load NUTS geo data using https://github.com/eurostat/Nuts2json

In [ ]:
%matplotlib inline
import geopandas as gpd

In [ ]:
def get_nuts_geopandas(year, nuts_level, scale, projection=3035, feature_type="nutsrg"):
    """Retrieves geo data of NUTS regions for specified year, level, scale and projection
    as a geopandas GeoDataFrame.

    Parameters
    ----------
    year : int
        Year for which to retrieve NUTS geo data. Available years: 2010, 2013, 2016.
    nuts_level : int
        The NUTS level to be displayed on the map. Available levels: 0, 1, 2, 3, from 
        national (`nuts_level`=0) to local level (`nuts_level`=3).
    scale : str
        The intended map scale factor. The larger the value, the stronger the simplification.
        For example, for a map width of 1000 pixels the scale 1:20M is the most suitable.
        Available scales: 1:10M, 1:20M, 1:60M
    projection : int, optional
        The coordinate reference system EPSG code, by default 3035.
        Two projections are provided: European LAEA projection (EPSG 3035) and
        web mercator (EPSG 3857). For statistical maps, it is strongly adviced to use an
        equal-area projection such as 3035. Geographic coordinates ETRS89 (EPSG 4258) and
        WGS84 (EPSG 4326) are also provided.
    feature_type : str, optional
        The feature type has to be specified, by default "nutsrg". For NUTS feature types
        (nutsrg and nutsbn), the `nuts_level` to be displayed on the map has to be specified.
        The available feature types can be found at: https://github.com/eurostat/Nuts2json.

    Returns
    -------
    geopandas.geodataframe.GeoDataFrame
        GeoDataFrame containing geo data according to `feature_type`
    """
    base_url="https://raw.githubusercontent.com/eurostat/Nuts2json/master"
    supported_years = {2010, 2013, 2016}
    if year not in supported_years:
        raise ValueError(f"year must be one of {supported_years}")
    supported_nuts_levels = {0, 1, 2, 3}
    if nuts_level not in supported_nuts_levels:
        raise ValueError(f"nuts_level must be one of {supported_nuts_levels}")
    supported_scales = {"10M", "20M", "60M"}
    if scale not in supported_scales:
        raise ValueError(f"scale must be one of {supported_scales}")
    supported_projections = {3035, 3857, 4258, 4326}
    if projection not in supported_projections:
        raise ValueError(f"projection must be one of {supported_projections}")
    
    url=f"{base_url}/{year}/{projection}/{scale}/{feature_type}_{nuts_level}.json"

    return gpd.read_file(url)

In [ ]:
europe_nuts = get_nuts_geopandas(2016, 1, "10M")

In [ ]:
europe_nuts.head()

In [ ]:
europe_nuts.loc[europe_nuts.na.str.contains("Karlsruhe")]

# Karlsruhe data

In [ ]:
from datenguidepy import get_regions

In [ ]:
regions = get_regions()

In [ ]:
KA_regions = regions.loc[regions.name.str.contains("Karlsruhe")]
KA_regions

overview of nuts levels can be found here: https://en.wikipedia.org/wiki/NUTS_statistical_regions_of_Germany

In [ ]:
KA_city_district_id = "08212"

In [ ]:
from datenguidepy import Query

In [ ]:
query = Query.region(KA_city_district_id)

### Add some statistics

First get some overview of available statistics and short descriptions

In [ ]:
from datenguidepy import get_statistics

In [ ]:
# return statistical descriptions in English
statistics = get_statistics(target_language = "en")
statistics.head()

In [ ]:
statistics.loc[statistics.short_description.str.contains("housing"), ["short_description"]]

`datenguidepy` provides convenience functionality to get an overview of available statistics:

In [ ]:
from datenguidepy import get_availability_summary 

In [ ]:
availability_result = get_availability_summary()
availability_KA = availability_result.loc[KA_city_district_id].copy()

In [ ]:
availability_KA.dropna().sort_values(by=["entries"], ascending=False)

example query of some statistics:

In [ ]:
year_range = list(range(1995, 2018))
stat1 = query.add_field("BETR01")
stat1.add_args({"year": year_range})
stat2 = query.add_field("ERW012")
stat2.add_args({"year": year_range})


In [ ]:
query.results(
    verbose_statistics = True,
    add_units = True,
)